# Diseño con la cuadricula

In [1]:
import pygame

# Configuración de la ventana
window_width = 1200
window_height = 800

# Configuración de la cuadrícula
grid_size = 10
grid_color = (200, 200, 200)  # Color de la cuadrícula en formato RGB

# Configuración de los elementos adicionales
road_width = 200
sidewalk_width = 100
green_color = (0, 255, 0)
white_color = (255, 255, 255)
grey_color = (128, 128, 128)
yellow_color = (255, 255, 0)

# Inicializar pygame y la ventana
pygame.init()
window = pygame.display.set_mode((window_width, window_height))
pygame.display.set_caption("Cuadrícula con elementos adicionales")

def draw_grid():
    # Dibujar líneas verticales
    for x in range(0, window_width, grid_size):
        pygame.draw.line(window, grid_color, (x, 0), (x, window_height))

    # Dibujar líneas horizontales
    for y in range(0, window_height, grid_size):
        pygame.draw.line(window, grid_color, (0, y), (window_width, y))

def draw_elements():
    # Dibujar los elementos adicionales
    pygame.draw.rect(window, white_color, (window_width/2 - road_width/2 - sidewalk_width, 0, sidewalk_width, window_height))
    pygame.draw.rect(window, white_color, (window_width/2 + road_width/2, 0, sidewalk_width, window_height))
    pygame.draw.rect(window, white_color, (0, window_height/2 - road_width/2 - sidewalk_width, window_width, sidewalk_width))
    pygame.draw.rect(window, white_color, (0, window_height/2 + road_width/2, window_width, sidewalk_width))

    pygame.draw.rect(window, grey_color, (window_width/2 - road_width/2, 0, road_width, window_height))
    pygame.draw.rect(window, grey_color, (0, window_height/2 - road_width/2, window_width, road_width))

    for y in range(0, window_height, 80):
        pygame.draw.rect(window, yellow_color, (window_width/2 - 5, y, 10, 40))
    for x in range(0, window_width, 80):
        pygame.draw.rect(window, yellow_color, (x, window_height/2 - 5, 40, 10))

# Bucle principal del programa
running = True
while running:
    # Manejo de eventos
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Obtener las coordenadas del cursor relativas a la cuadrícula
    mouse_x, mouse_y = pygame.mouse.get_pos()
    grid_x = (mouse_x // grid_size) * grid_size
    grid_y = (mouse_y // grid_size) * grid_size

    # Rellenar la ventana con color verde
    window.fill(green_color)

    # Dibujar los elementos adicionales
    draw_elements()

    # Dibujar la cuadrícula
    draw_grid()

    # Mostrar las coordenadas del cursor en la cuadrícula
    font = pygame.font.Font(None, 30)
    text = font.render(f"({grid_x}, {grid_y})", True, white_color)
    window.blit(text, (10, 10))

    # Actualizar la ventana
    pygame.display.flip()

# Cerrar pygame al salir
pygame.quit()


pygame 2.4.0 (SDL 2.26.4, Python 3.10.11)
Hello from the pygame community. https://www.pygame.org/contribute.html


# PRUEBAS DE COLISION

In [12]:
import pygame

# Constantes
WIDTH = 800
HEIGHT = 600
CAR_SPEED = 1
FPS = 30

# Inicialización de Pygame
pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
clock = pygame.time.Clock()

# Definición de colores
WHITE = (255, 255, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255, 20)

# Clase Carro
class Carro:
    def __init__(self, x, y, width, height, color, direction):
        self.rect = pygame.Rect(x, y, width, height)
        self.color = color
        self.direction = direction
        self.has_collision = False

    def move(self, semaforo, carros):
        if semaforo.is_red() and self.has_collision:
            return  # No hacer nada si hay una colisión y el semáforo está en rojo

        if self.direction == 'up':
            self.rect.y -= CAR_SPEED
        elif self.direction == 'down':
            self.rect.y += CAR_SPEED
        elif self.direction == 'left':
            self.rect.x -= CAR_SPEED
        elif self.direction == 'right':
            self.rect.x += CAR_SPEED

        # Verificar colisión con otros carros
        for carro in carros:
            if carro != self and self.rect.colliderect(carro.rect):
                self.has_collision = True
                if self.direction == 'up':
                    self.rect.y = carro.rect.y - self.rect.height
                elif self.direction == 'down':
                    self.rect.y = carro.rect.y + carro.rect.height
                elif self.direction == 'left':
                    self.rect.x = carro.rect.x - self.rect.width
                elif self.direction == 'right':
                    self.rect.x = carro.rect.x + carro.rect.width
                break
        else:
            self.has_collision = False

    def draw(self, screen):
        pygame.draw.rect(screen, self.color, self.rect)


# Clase Semaforo
class Semaforo:
    def __init__(self, x, y, size):
        self.light_color = RED
        self.collider_color = BLUE
        self.light_rect = pygame.Rect(x, y, size, size)
        self.collider_rect = pygame.Rect(x - size, y - size, size * 3, size * 3)

    def change_color(self):
        if self.light_color == RED:
            self.light_color = GREEN
        else:
            self.light_color = RED

    def is_red(self):
        return self.light_color == RED

    def draw(self, screen):
        pygame.draw.rect(screen, self.collider_color, self.collider_rect)
        pygame.draw.rect(screen, self.light_color, self.light_rect)


# Crear objetos
carros = []
carros.append(Carro(WIDTH // 2 - 20, HEIGHT - 80, 40, 20, WHITE, 'up'))
carros.append(Carro(WIDTH // 2 - 20, HEIGHT - 120, 40, 20, WHITE, 'up'))
carros.append(Carro(WIDTH // 2 - 20, HEIGHT - 160, 40, 20, WHITE, 'up'))

semaforo = Semaforo(WIDTH // 2 - 20, HEIGHT // 2 - 20, 40)

# Bucle principal del juego
running = True
while running:
    # Manejo de eventos
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Lógica del juego
    for carro in carros:
        carro.move(semaforo, carros)
        if carro.rect.colliderect(semaforo.collider_rect):
            carro.has_collision = True
        else:
            carro.has_collision = False
            
    # Cambiar color del semáforo cada segundo
    if pygame.time.get_ticks() % 500 == 0:
        semaforo.change_color()

    # Dibujar en la pantalla
    screen.fill((0, 0, 0))  # Limpiar pantalla
    for carro in carros:
        carro.draw(screen)
    semaforo.draw(screen)
    pygame.display.flip()  # Actualizar pantalla

    clock.tick(FPS)

# Salir del juego
pygame.quit()
